# CPSC 330 Lecture 10

#### Lecture plan

- 👋
- **Turn on recording**
- Announcements (5 min)
- Dataset of the week: regression! (15 min)
- Regression score functions: mean squared error and R^2 (10 min)
- Linear regression intro (5 min)
- Break (5 min)
- Exploring `alpha` (5 min)
- Transforming the targets (20 min)
- Ensembling with regression (5 min)

In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from pandas_profiling import ProfileReport

In [3]:
plt.rcParams['font.size'] = 16

## Announcements

Homework assignments:

- hw4 deadline passed
- hw5 coming today, due Monday at 11:59pm
- After that a 2-week break from hw 

Midterm:

- a

# TODO

- make some feature; why not price per square foot? (because it uses the target!)
- Also: MoSold (month sold) feature - to discuss?
- https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html


## Data set of the week (20 min)

This week in lecture we will be focussing on the [Kaggle House Prices dataset](https://www.kaggle.com/c/home-data-for-ml-course/) with house prices from Ames, Iowa. As usual, to run this notebook you'll need to download the data. Unzip the data. Rename `train.csv` to `housing.csv` and move it into the data directory of this repo. (For this dataset, train and test have already been separated. The "test" data they provide is actually what we call deployment. They only provide the X, not the y. So we couldn't actually use Kaggle's test set as our test set, since we need the y values for that.)

In [21]:
df = pd.read_csv("data/housing.csv", index_col=0)

In [22]:
df_train, df_test = train_test_split(df, random_state=123)

In [23]:
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1447,20,RL,NaN,26142,Pave,NaN,IR1,Lvl,AllPub,CulDSac,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,157900
1124,20,RL,50.0,9405,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2009,WD,Normal,118000
187,80,RL,NaN,9947,Pave,NaN,IR1,Lvl,AllPub,CulDSac,...,0,NaN,GdPrv,NaN,0,6,2009,WD,Normal,173000
1021,20,RL,60.0,7024,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,176000
68,20,RL,72.0,10665,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2007,WD,Normal,226000


- Here, the target is `SalePrice`. Note that this is numeric, not categorical.
- In this case, we call the task **regression** (as opposed to classification).

In [24]:
df_train.shape

(1095, 80)

Above: this dataset has very few examples, but a lot of features.

In [25]:
# df_train.info()

#### Step 1: `pandas_profiler`

In [26]:
profile = ProfileReport(df_train, minimal=True)

In [27]:
profile.to_notebook_iframe();

#### Types of features

- How does pandas profiling figure out the data type?
- You can look at the Python data type and say floats are numeric, strings are categorical. 
  - However, in doing so you would miss out on various subtleties such as some of the string features being ordinal rather than truly categorical. 
  - Also, it will think free text is categorical. 

This is a good time to look at the `data_description.txt` provided with the dataset.


- There are a bunch of ordinal features using the same scale: excellent, good, average, etc.
  - These I'm calling `ordinal_features_reg`.
- There are a bunch more ordinal features using different scales.
  - These I'm calling `ordinal_features_oth`
  - To save time I'm ignoring the ordinality and just encoding them as categorical.

In [45]:
X_train = df_train.drop(columns=['SalePrice'])
y_train = df_train['SalePrice']

X_test = df_test.drop(columns=['SalePrice'])
y_test = df_test['SalePrice']

In [53]:
numeric_features     = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 
                        'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
                        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 
                        'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 
                        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
                        'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold']
ordinal_features_reg = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
                        'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
ordinal_features_oth = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                        'Functional',  'Fence']
categorical_features = list(set(X_train.columns) - set(numeric_features) - set(ordinal_features_reg))

#all_features = numeric_features + ordinal_features_reg + categorical_features + ordinal_features_oth

ordering = ['Po', 'Fa', 'TA', 'Gd', 'Ex'] # if N/A it will just impute something, per below

#### Step 2: build Pipelines for each feature type

We'll use the `make_pipeline` syntax in sklearn, which doesn't require us to specify a name each time.

In [54]:
numeric_preprocessing = make_pipeline(SimpleImputer(strategy='median'), 
                                      StandardScaler())

In [55]:
ordinal_preprocessing = make_pipeline(SimpleImputer(strategy='most_frequent'), 
                                      OrdinalEncoder(categories=[ordering]*len(ordinal_features_reg)))

In [85]:
categorical_preprocessing = make_pipeline(SimpleImputer(strategy='constant', fill_value="?"),
                                          OneHotEncoder(handle_unknown='ignore', sparse=False))

- We'll just use `SimpleImputer` again.
  - In reality we'd want to go through this more carefully.
  - We may also want to drop some columns that are almost entirely missing.
  - We could also check for outliers, and do other EDA.
- Also, note how `make_pipeline` gives each step a default name:

In [86]:
ordinal_preprocessing

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex']]))])

Now we'll build the `ColumnTransformer`:

In [87]:
preprocessing = ColumnTransformer([
    ('numeric', numeric_preprocessing, numeric_features),
    ('ordinal', ordinal_preprocessing, ordinal_features_reg),
    ('categorical', categorical_preprocessing, categorical_features)
])

In the hw4 question I accidentally deleted, I asked you to try this out on your training data,
just to see if you could get the columns lined up:

In [88]:
preprocessing.fit(X_train);

In [89]:
ohe_columns = list(preprocessing.named_transformers_['categorical'].named_steps['onehotencoder'].get_feature_names(categorical_features))
new_columns = numeric_features + ordinal_features_reg + ohe_columns

In [93]:
X_train_enc = pd.DataFrame(preprocessing.transform(X_train), index=X_train.index, columns=new_columns)

In [94]:
X_train_enc.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,Electrical_SBrkr,Utilities_AllPub,Utilities_NoSeWa,Street_Grvl,Street_Pave,Fence_?,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
Id,,,,,,,,,,,,,,,,,,,,,
1447,-0.046315,1.654400,-0.775646,1.255836,-0.282035,-1.105566,0.491436,0.327430,-0.28498,0.064219,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1124,-0.888437,-0.093394,-0.775646,3.019968,-0.773017,1.117128,-0.569906,-0.942714,-0.28498,0.297501,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
187,-0.046315,-0.036795,0.647021,-0.508295,0.634466,0.295697,-0.569906,0.365985,-0.28498,0.023451,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1021,-0.420591,-0.342035,-1.486980,-0.508295,1.125448,0.972169,-0.569906,1.250588,-0.28498,-1.038776,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
68,0.140824,0.038184,0.647021,-0.508295,1.059984,0.875531,0.367894,1.227027,-0.28498,-0.286837,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


So, the OHE took us from 

In [135]:
X_train.shape

(1095, 79)

In [136]:
X_train_enc.shape

(1095, 292)

79 columns to 292 columns!

#### Step 3: `DummyRegressor`

In [97]:
dummy = DummyRegressor()

In [98]:
pd.DataFrame(cross_validate(dummy, X_train, y_train, return_train_score=True))

,fit_time,score_time,test_score,train_score
0,0.002502,0.000392,-0.011348,0.0
1,0.001542,0.000311,-0.015114,0.0
2,0.001188,0.000294,-0.001615,0.0
3,0.001155,0.000295,-0.000229,0.0
4,0.001735,0.000334,-0.000594,0.0


Wait, a negative score??

## Regression score functions: mean squared error and R^2 (10 min)

We aren't doing classification anymore, so we can't just check for equality:

In [103]:
dummy.fit(X_train, y_train);

In [104]:
dummy.predict(X_train) == y_train

Id
1447    False
1124    False
187     False
1021    False
68      False
        ...  
1042    False
1123    False
1347    False
1407    False
1390    False
Name: SalePrice, Length: 1095, dtype: bool

In [105]:
y_train.values

array([157900, 118000, 173000, ..., 262500, 133000, 131000])

In [106]:
dummy.predict(X_train)

array([180201.98812785, 180201.98812785, 180201.98812785, ...,
       180201.98812785, 180201.98812785, 180201.98812785])

- We need a score that reflect how right/wrong each prediction is.

#### Mean squared error (MSE)

A common measure is mean squared error:

In [107]:
preds = dummy.predict(X_train)

In [108]:
np.mean((y_train - preds)**2)

6396820839.792553

This is also implemented in sklearn:

In [111]:
mean_squared_error(y_train, preds)

6396820839.792553

- Perfect predictions would have MSE=0. 

In [112]:
mean_squared_error(y_train, y_train)

0.0

- But is the above score good or bad? 
  - It depends on the scale of the targets.
  - If we were working in cents instead of dollars, our MSE would be 10,000x ($100^2$) higher!

In [113]:
np.mean((y_train*100 - preds*100)**2)

63968208397925.53

- A common score is the $R^2$. 
- You can [read about it](https://en.wikipedia.org/wiki/Coefficient_of_determination) if interested.
- Intuition: mean squared error, but flipped (higher is better), and normalized so the max is 1.
- Key points:
  - The maximum is 1 for perfect predictions
  - Negative values are very bad: "worse than `DummyRegressor`" (very bad)

In [116]:
r2_score(y_train, preds)

0.0

In [117]:
r2_score(y_train, y_train)

1.0

This is the score that sklearn uses by default when you call `score()`:

In [119]:
dummy.score(X_train, y_train)

0.0

(optional) Warning: MSE is "reversible" but $R^2$ is not:

In [120]:
mean_squared_error(y_train, preds)

6396820839.792553

In [121]:
mean_squared_error(preds, y_train)

6396820839.792553

In [124]:
r2_score(y_test, dummy.predict(X_test))

-0.0013723866815522623

In [125]:
r2_score(dummy.predict(X_test), y_test)

-7.129310472299063e+30

- When you call `fit` it minimizes MSE / maximizes $R^2$ (or something like that) by default.
- Just like in classification, this isn't always what you want!!
- More on this later.

## Linear regression intro (5 min)

- Linear regression is one of the most basic and popular ML/statistical techniques. 
- We already saw logistic regression for classification - these are close cousins.
- Both are very _interpretable_.
- For more depth, see CPSC 340, STAT 306, and various other STAT courses.
- In scikit-learn, we can do:

In [128]:
LinearRegression();

However, I am going to recommend always using:

In [129]:
Ridge();

`Ridge` is more flexible than `LinearRegression` because it has a hyperparameter `alpha`:

In [130]:
lr_vanilla = make_pipeline(preprocessing, LinearRegression())
lr_vanilla.fit(X_train, y_train);

In [151]:
lr_vanilla_preds = lr_vanilla.predict(X_test)
lr_vanilla_preds[:10]

array([ 2.24673250e+05,  5.64907500e+04,  1.40039250e+05,  2.57581250e+05,
        1.30109250e+05,  2.45723250e+05,  3.29241250e+05, -9.63528122e+13,
        1.39126750e+05,  1.31254250e+05])

In [152]:
lr_vanilla_preds.max()

1816716012695273.2

In [153]:
lr_vanilla_preds.min()

-336572025674397.25

- One prediction is for $\$10^{15}$ !!
- One prediction is for $ - \$10^{15}$ !!!!
- This happened because we have "collinear features" and there are "numerical issues" (see also CPSC 302).
- The coefficients got extremely huge:

In [148]:
coefs = lr_vanilla.named_steps['linearregression'].coef_

In [149]:
coefs.min()

-1.6662898194204134e+16

In [150]:
coefs.max()

1.6066143091001834e+16

`Ridge` "solves" this issue:

In [154]:
lr = make_pipeline(preprocessing, Ridge())
lr.fit(X_train, y_train);

In [156]:
lr_preds = lr.predict(X_test)
lr_preds[:10]

array([222545.43672749,  77055.21683115, 133676.81431636, 248214.02568334,
       123239.48200752, 206608.39854061, 338498.31839006, 116584.73477258,
       145709.98643343, 120994.59172911])

In [157]:
lr_preds.max()

485186.1011329584

In [158]:
lr_preds.min()

29933.09133236739

- So, TL;DR, never use `LinearRegression`, always use something else, such as `Ridge`.
  - Unless you know what you're doing, but then use [statsmodels](https://www.statsmodels.org/stable/index.html) or R instead of sklearn. 
- This also allows us to use `handle_unknown='ignore'` in our OHE.
- For more details, see STAT 306, etc.

So, let's see how well this model does:

In [159]:
def cross_validate_std(*args, **kwargs):
    res = pd.DataFrame(cross_validate(*args, **kwargs))
    test_score_std = res["test_score"].std()
    train_score_std = res["train_score"].std()
    res_mean = res.mean()
    res_mean["std_test_score"] = test_score_std
    res_mean["std_train_score"] = train_score_std
    return res_mean

In [160]:
cross_validate_std(lr, X_train, y_train, cv=10, return_train_score=True)

fit_time           0.051157
score_time         0.012856
test_score         0.756365
train_score        0.923955
std_test_score     0.244919
std_train_score    0.004777
dtype: float64

The std of the test score is very high:

In [161]:
pd.DataFrame(cross_validate(lr, X_train, y_train, cv=10, return_train_score=True))

,fit_time,score_time,test_score,train_score
0,0.046411,0.013827,0.714842,0.928843
1,0.101983,0.016429,0.853926,0.920657
2,0.064470,0.012895,0.868423,0.922619
3,0.061216,0.015063,0.852679,0.922514
4,0.052639,0.012838,0.773667,0.922014
5,0.052297,0.012908,0.806447,0.926552
6,0.051542,0.012310,0.851162,0.921801
7,0.050210,0.012371,0.858105,0.921226
8,0.051270,0.012481,0.076819,0.934708
9,0.053398,0.012920,0.907581,0.918615


It seems to do terribly on one fold, not sure why.

## Break (5 min)

<br><br>

## Exploring `alpha` (5 min)

- `Ridge` has a hyperparameter `alpha`.
- This is like `C` in `LogisticRegression` but, annoyingly, `alpha` is the _inverse_ of `C`.
  - That is, large `C` is like small `alpha` and vice versa.
- We can (approximately) recover the original `LinearRegression` by setting `alpha=0` (but I don't recommend this).

Let's compare with the random forest (the regression version!):

In [ ]:
lr = Ridge()
lr.fit(X_train_imp_encode, y_train_log);

In [ ]:
print("Training MAPE: %.1f%%" % mape(y_train, np.exp(lr.predict(X_train_imp_encode))))

In [ ]:
print("Validation MAPE: %.1f%%" % mape(y_valid, np.exp(lr.predict(X_valid_imp_encode))))

In [ ]:
rf = RandomForestRegressor(random_state=111)
rf.fit(X_train_imp_encode, np.log(y_train));

In [ ]:
print("Training MAPE: %.1f%%" % mape(y_train, np.exp(rf.predict(X_train_imp_encode))))

In [ ]:
print("Validation MAPE: %.1f%%" % mape(y_valid, np.exp(rf.predict(X_valid_imp_encode))))

- So far, the linear model seems to be doing better. 
- Note that it's also a _simpler_ model.
- Like logistic regression, the intuition is that "the complexity grows as you add more features". 

- The `alpha` hyperparameter controls the fundamental tradeoff as usual.
  - Smaller `alpha`: lower training error.
  - Larger `alpha`: lower approximation error (hopefully). 
- General intuition: larger `alpha` leads to **smaller coefficients**. 
  - Smaller coefficients mean the predictions are less sensitive to changes in the data.
  - Hence less chance of overfitting (seeing big dependencies when you shouldn't).
- Let's test this out:

In [ ]:
lr = Ridge(alpha=0.01)
lr.fit(X_train_imp_encode, y_train_log);
np.max(np.abs(lr.coef_))

In [ ]:
lr = Ridge(alpha=1)
lr.fit(X_train_imp_encode, y_train_log);
np.max(np.abs(lr.coef_))

In [ ]:
lr = Ridge(alpha=1000)
lr.fit(X_train_imp_encode, y_train_log);
np.max(np.abs(lr.coef_))

- Note that it will not make all the coefficients smaller in the same proportion!
- The order of the coefficients might change. 

In [ ]:
lr = Ridge(alpha=0.01)
lr.fit(X_train_imp_encode, y_train_log);
df_train_imp_encode.columns[np.argmax(np.abs(lr.coef_))]

In [ ]:
lr = Ridge(alpha=1)
lr.fit(X_train_imp_encode, y_train_log);
df_train_imp_encode.columns[np.argmax(np.abs(lr.coef_))]

In [ ]:
lr = Ridge(alpha=100)
lr.fit(X_train_imp_encode, y_train_log);
df_train_imp_encode.columns[np.argmax(np.abs(lr.coef_))]

Let's try tuning `alpha`:

In [ ]:
alphas = 10.0**np.arange(-1.5,4.5,0.5)
train_errs = []
valid_errs = []
for alpha in alphas:

    lr = Ridge(alpha=alpha)
    lr.fit(X_train_imp_encode, y_train_log);
    train_errs.append(mape(y_train, np.exp(lr.predict(X_train_imp_encode))))
    valid_errs.append(mape(y_valid, np.exp(lr.predict(X_valid_imp_encode))))

In [ ]:
plt.semilogx(alphas, train_errs, label="train");
plt.semilogx(alphas, valid_errs, label="valid");
plt.legend();
plt.xlabel('alpha');
plt.ylabel('MAPE');

In [ ]:
best_alpha = alphas[np.argmin(valid_errs)]
best_alpha

In [ ]:
train_errs = []
valid_errs = []
for alpha in alphas:

    lr = Ridge(alpha=alpha)
    lr.fit(X_train_imp_encode, y_train);
    train_errs.append(np.sqrt(mean_squared_error(y_train, lr.predict(X_train_imp_encode))))
    valid_errs.append(np.sqrt(mean_squared_error(y_valid, lr.predict(X_valid_imp_encode))))

In [ ]:
plt.semilogx(alphas, train_errs, label="train");
plt.semilogx(alphas, valid_errs, label="valid");
plt.legend();
plt.xlabel('alpha');
plt.ylabel('MSE');

In [ ]:
best_alpha = alphas[np.argmin(valid_errs)]
best_alpha

- These are interesting curves, because the validation error is less than the training error.
- In short, that is because when `alpha` is large we are no longer directly minimizing training error.
- It seems `alpha=100` is the best choice here. 

In [ ]:
lr = Ridge(alpha=100)
lr.fit(X_train_imp_encode, y_train_log);

In [ ]:
print("Training MAPE: %.1f%%" % mape(y_train, np.exp(lr.predict(X_train_imp_encode))))

In [ ]:
print("Validation MAPE: %.1f%%" % mape(y_valid, np.exp(lr.predict(X_valid_imp_encode))))

- I'm sure one could do a lot better on this dataset, but 9% MAPE is a start!

## Transforming the targets (15 min)

Let's try something more serious than `DummyRegressor`:

In [ ]:
rf = RandomForestRegressor(random_state=111)
rf.fit(X_train_imp_encode, y_train);

In [ ]:
rf.score(X_train_imp_encode, y_train)

In [ ]:
rf.score(X_valid_imp_encode, y_valid)

In [ ]:
mse_valid = mean_squared_error(y_valid, rf.predict(X_valid_imp_encode))
mse_valid

- In this case, we are actually interested in the original units of dollars.
- MSE is in units of "squared dollars"
- We can take the square root of the this to get back to dollars:

In [ ]:
rmse_valid = np.sqrt(mse_valid)
rmse_valid

So, on average we're off by about \$25000. Is this good?

<br><br><br><br><br><br>

- For a house worth \\$500k, it seems reasonable! That's 5% error.
- For a house worth \\$50k, that is terrible. It's 50% error.

In [ ]:
plt.hist(y_train, bins=100);

- Indeed, we have both of these cases in our dataset.
- Can we compute percent error?

In [ ]:
pred_train = rf.predict(X_train_imp_encode)
pred_valid = rf.predict(X_valid_imp_encode)

In [ ]:
percent_errors = (pred_train - y_train)/y_train
percent_errors

- These are both positive (predict too high) and negative (predict too low).
- We can look at the absolute percent error:

In [ ]:
np.abs(percent_errors)

And, like MSE, we can take the average over examples. This is called **mean absolute percent error (MAPE)**.

In [ ]:
def mape(true, pred):
    return 100.*np.mean(np.abs((pred - true)/true))

In [ ]:
mape(y_train, pred_train)

In [ ]:
mape(y_valid, pred_valid)

- Ok, this is quite interpretable.
- On average, we have 11% error. Good to know. 

- ... but wait a minute, why are we minimizing MSE if we care about MAPE??
- When minimizing MSE, **the expensive houses will dominate** because they have the biggest error.
- The model would rather do \\$50 k worse on a cheap place and \\$60k better on an expensive place.
  - But this would make the MAPE much worse!
- Key idea: **log transform the targets**.
  - That is, transform $y\rightarrow \log(y)$.
- Why?
- Let's assume we have two values, the prediction and the true value.
- We log transform them and look at the squared error:

$$\begin{align}(\log y_\text{pred} - \log y_\text{true})^2 =\left(\log \frac{y_\text{pred}}{y_\text{true}}\right)^2  \end{align}$$

But what is the absolute percent error? It is

$$\left| \frac{y_\text{pred} - y_\text{true}}{y_\text{true}} \right| = \left| \frac{y_\text{pred}}{y_\text{true}} -1 \right|$$

This is a bit hand-wavy (and maybe I'll nail this down later), but for now: both are minimized (equal to zero) when $\frac{y_\text{pred}}{y_\text{true}}=1$, so both are trying to make $\frac{y_\text{pred}}{y_\text{true}}$ close to $1$.

In [ ]:
plt.hist(y_train, bins=100);

In [ ]:
plt.hist(np.log(y_train), bins=100);

In [ ]:
rf_log = RandomForestRegressor(random_state=111)
rf_log.fit(X_train_imp_encode, np.log(y_train));

In [ ]:
rf_log.predict(X_train_imp_encode)

These are log predictions. We can `exp` them to get back to dollars:

In [ ]:
pred_train_log_exp = np.exp(rf_log.predict(X_train_imp_encode))
pred_valid_log_exp = np.exp(rf_log.predict(X_valid_imp_encode))

In [ ]:
pred_train_log_exp

How is the MSE on the training set?

In [ ]:
mean_squared_error(pred_train, y_train) # ORIGINAL model

In [ ]:
mean_squared_error(pred_train_log_exp, y_train) # log transformed training

- The MSE got worse
  - That makes sense because we're no longer optimizing for MSE.
- Let's look at the MAPE.

In [ ]:
mape(y_train, pred_train)

In [ ]:
mape(y_train, pred_train_log_exp)

- The MAPE got better!
- We can do the same for the validation set:

In [ ]:
mape(y_valid, pred_valid)

In [ ]:
mape(y_valid, pred_valid_log_exp)

- Here we get a small benefit, but it could be very large in some cases.
- Also, the model's interpretation can often be better in the log transformed case.
- For linear regression: 
  - 1 more bedroom increases price by \\$50K vs
  - 1 more bedroom increases price by 5%.


- Note that this assumes the $y$-values are positive, which is true in this case.
  - There is still a problem if one of the $y$-values is zero, so it's common to do $\log(1+y)$ instead of $\log(y)$.
  - There is even a numpy function to do this for you: [`log1p`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log1p.html) (better handing of floating point issues for small numbers -- not too relevant to us here).

## Ensembling with Regression (5 min)
mention how ensembling works with regression

1. show that it's the actual average
2. show how the LR coefs are now Ridge coefs.
3. Stacking doesn't have to use predict_proba - well, there is no predict_proba!

## Linear regression True/False (Piazza)

1. If the first coefficient is 5, that means increasing your first feature by 1 increases the prediction by 5.
2. Since the `PoolArea` has a positive coefficient, expanding my pool will get me a higher price when I sell my house.
3. Larger values of `alpha` are probably more useful when I have lots of features.
4. log-transforming the targets (and re-fitting) is equivalent to log-transforming the coefficients. 
5. In regression, one should use MAPE instead of MSE when relative (percent) error matters more than absolute error.